# TTT (Text-to-Text) Client

## 1. Setup

1. Clone the repository.

```bash
git clone https://github.com/kakkoii1337/gai-sdk
```

2. Initialize the SDK.

```bash
gai init
```

3. Download the Model.

```bash
gai pull llamacpp-dolphin
```


## 2. Chat

In [1]:
from gai.ttt.client.ttt_client import TTTClient
client = TTTClient(config={
    "client_type":"gai",
    "url":"http://localhost:12031/gen/v1/chat/completions"
    })
response=client(messages=[
        {"role":"user","content":"tell me a one sentence story"},
        {"role":"assistant","content":""}
    ],
    stream=False
    )
print(response.extract())

{'type': 'content', 'content': 'Once upon a time, a little boy found a magical pendant that granted his every wish, but it came at a cost he never expected.'}


## 3. Stream

In [2]:
response=client(messages=[
    {"role":"user","content":"tell me a one sentence story"},
    {"role":"assistant","content":""}
    ],stream=True)
for chunk in response:
    chunk = chunk.extract()
    print(chunk, end="", flush=True)


A lonely astronaut found a message in a bottle on an alien planet, which read, "Help, I'm trapped in this universe too."

## 4. Tool Call

In [3]:
messages = [
    {"role":"user","content":"What is the current time in Singapore?"},
    {"role":"assistant","content":""}
]
tool_choice="required"
tools = [
    {
        "type": "function",
        "function": {
            "name": "google",
            "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_query": {
                        "type": "string",
                        "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                    }
                },
                "required": ["search_query"]
            }
        }
    }
]
response=client(messages=messages,stream=False,tool_choice=tool_choice,tools=tools)
print(response.extract())


{'type': 'function', 'name': 'google', 'arguments': '{"search_query": "current time in Singapore"}'}


## 5. JSON Schema

In [4]:
# Define Schema
from pydantic import BaseModel
class Book(BaseModel):
    title: str
    summary: str
    author: str
    published_year: int

response=client(
    messages=[
        {"role":"user","content":"""Foundation is a science fiction novel by American writer 
            Isaac Asimov. It is the first published in his Foundation Trilogy (later 
            expanded into the Foundation series). Foundation is a cycle of five 
            interrelated short stories, first published as a single book by Gnome Press 
            in 1951. Collectively they tell the early story of the Foundation, 
            an institute founded by psychohistorian Hari Seldon to preserve the best 
            of galactic civilization after the collapse of the Galactic Empire.
            """},
        {"role":"assistant","content":""}
        ],
    json_schema=Book.schema(),
    tool_choice="none",
    timeout=50,
    stream=False
    )
print(response.extract())

/tmp/ipykernel_1620662/3905909139.py:21: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  json_schema=Book.schema(),


{'type': 'content', 'content': '{ "title": "Foundation", "summary": "Foundation is a science fiction novel by American writer Isaac Asimov. It is the first published in his Foundation Trilogy (later expanded into the Foundation series). Foundation is a cycle of five interrelated short stories, first published as a single book by Gnome Press in 1951. Collectively they tell the early story of the Foundation, an institute founded by psychohistorian Hari Seldon to preserve the best of galactic civilization after the collapse of the Galactic Empire." ,"author": "Isaac Asimov", "published_year": 1951}'}
